# <center>Data Mining project: Discover and describe areas of interest<br> and events from geo-located data</center>

## #0: Import Dataset and Libraries

## Setting Environment

In [1]:
! python -m venv dataMiningEnv
#Activate windows
#! dataMiningEnv\Scripts\activate.bat

#Activate mac
! source dataMiningEnv/bin/activate


## Importing Libraries

- ### Installs

In [2]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3
# Language processing
! pip install nltk
! python -m nltk.downloader popular # popular functions


ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5; 1.12.1 Requires-Python >=3.7, !=3.9.7; 1.12.1rc1 Requires-Python >=3.7, !=3.9.7; 1.12.2 Requires-Python >=3.7, !=3.9.7; 1.12.2rc1 Requires-Python >=3.7, !=3.9.7; 1.12.2rc2 Requires-Python >=3.7, !=3.9.7; 1.13.0 Requires-Python >=3.7, !=3.9.7; 1.13.0rc1 Requires-Python >=3.7, !=3.9.7; 1.13.0rc2 Requires-Python >=3.7, !=3.9.7; 1.14.0 Requires-Python >=3.7, !=3.9.7; 1.14.0rc1 Requires-Python >=3.7, !=3.9.7; 1.14.1 Requires-Python >=3.7, !=3.9.7; 1.14.1rc1 Requires-Python >=3.7, !=3.9.7; 1.15.0 Requires-Python >=3.7, !=3.9.7; 1.15.1 Requires-Python >=3.7, !=3.9.7; 1.15.2 Requires-Python >=3.7, !=3.9.7; 1.15.2rc1 Requires-Python >=3.7, !=3.9.7; 1.16.0 Requires-Python >=3.7, !=3.9.7; 1.17.0 Requires-Python >=3.7, !=3.9.7; 1.18.0 Requires-Python >=3.7, !=3.9.7; 1.18.1 Requires-Python >=3.7, !=3.9.7; 1.18.1rc1 Requires-Python >=3.7, !=3.9.7; 1.19.0 Requires-Python >=3.7, !=3.9.7; 1.20.0 

- ### Imports

In [3]:
import pandas as pd

import nltk


/Users/saadelghissassi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/saadelghissassi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Importing Data

In [ ]:
# Charger les données avec low_memory=False pour éviter les avertissements
data = pd.read_csv("data/flickr_data2.csv", sep=",", low_memory=False)

## #1: Discovering areas of interests using clustering

## Understanding the Data

--Write something to describe this part of the report--

## Data clearing and preparation

In [ ]:
# Étape 1 : Nettoyage initial des colonnes
# Supprimer les espaces supplémentaires dans les noms des colonnes
data.columns = data.columns.str.strip()

# Convertir les colonnes temporelles en numériques
# Liste des colonnes temporelles
time_columns = [
    'date_taken_minute', 'date_taken_hour', 'date_taken_day',
    'date_taken_month', 'date_taken_year',
    'date_upload_minute', 'date_upload_hour', 'date_upload_day',
    'date_upload_month', 'date_upload_year'
]

# Convertir chaque colonne en int64, remplacer les erreurs par 0
for col in time_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype('int64')

In [ ]:
# Étape 2 : Suppression des doublons basés sur l'identifiant unique
# Sauvegarder les doublons pour audit futur
data[data['id'].duplicated(keep='first')].sort_values("id").to_csv(
    "data/parsed_lines/duplicatedId.csv", index=False
)

# Supprimer les doublons
data = data[~data['id'].duplicated(keep='first')]

In [ ]:
# Étape 3 : Gestion des colonnes inutilisées ou corrompues
# Identifier les colonnes inutiles
unused_columns = ["Unnamed: 16", "Unnamed: 17", "Unnamed: 18", 
                  "date_upload_minute", "date_upload_hour", "date_upload_day", 
                  "date_upload_month", "date_upload_year"]

# Sauvegarder les données corrompues
corrupted_data = data[data[unused_columns].notnull().any(axis=1)]
corrupted_data.to_csv("data/parsed_lines/corrupted_data.csv", index=False)

# Supprimer les colonnes inutilisées et les lignes corrompues
data = data.drop(columns=unused_columns)

In [ ]:
# Étape 4 : Nettoyage des coordonnées GPS
# Définir les limites géographiques de Lyon
lyon_lat_min, lyon_lat_max = 45.69, 45.85
lyon_lon_min, lyon_lon_max = 4.78, 4.92

# Filtrer les données pour garder uniquement les points dans Lyon
data = data[
    (data['latitude'] >= lyon_lat_min) & 
    (data['latitude'] <= lyon_lat_max) &
    (data['longitude'] >= lyon_lon_min) &
    (data['longitude'] <= lyon_lon_max)
]

In [ ]:
# Étape 5 : Gestion des valeurs manquantes
# Supprimer les lignes avec des valeurs manquantes dans latitude/longitude
data = data.dropna(subset=['latitude', 'longitude'])

# Remplir les valeurs manquantes pour les colonnes textuelles par des chaînes vides
data['tags'] = data['tags'].fillna('')
data['title'] = data['title'].fillna('')

In [ ]:
# Étape 6 : Suppression des doublons géographiques et temporels
# Identifier les doublons basés sur latitude, longitude, et année
data = data.drop_duplicates(subset=['latitude', 'longitude', 'date_taken_year'])

In [ ]:
# Étape 7 : Filtrage des dates incohérentes
# Garder uniquement les dates raisonnables (entre 2000 et 2025)
data = data[
    (data['date_taken_year'] >= 2000) &
    (data['date_taken_year'] <= 2025)
]

In [ ]:
# Étape 8 : Nettoyage des colonnes textuelles
# Fonction pour nettoyer le texte
def clean_text(text):
    # Supprimer les caractères spéciaux et passer en minuscules
    return re.sub(r'[^a-zA-Z0-9, ]', '', text).lower()

# Appliquer le nettoyage sur les colonnes textuelles
data['tags'] = data['tags'].apply(clean_text)
data['title'] = data['title'].apply(clean_text)

In [ ]:
# Étape 9 : Normalisation des coordonnées GPS
# Normaliser latitude et longitude pour les algorithmes de clustering
scaler = StandardScaler()
data[['latitude', 'longitude']] = scaler.fit_transform(data[['latitude', 'longitude']])

In [ ]:
# Résumé final des données nettoyées
print("Données nettoyées :")
print(data.info())

In [ ]:
# Sauvegarder les données nettoyées pour les prochaines étapes
data.to_csv("data/cleaned_flickr_data.csv", index=False)